In [6]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import gzip, pickle
from sklearn.preprocessing import OneHotEncoder

In [5]:
%matplotlib inline

In [23]:
def load_data():
    with gzip.open('/home/aaditya/.keras/datasets/mnist.pkl.gz', 'rb') as f:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    x_train, y_train = train_set
    x_val, y_val = valid_set
    x_test, y_test = test_set

    x_train = np.array(x_train, dtype='float32').reshape(-1,28,28)
    x_val = np.array(x_val, dtype='float32').reshape(-1,28,28)
    x_test = np.array(x_test, dtype='float32').reshape(-1,28,28)

    y_train = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_train.reshape(-1,1))
    y_val = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_val.reshape(-1,1))
    y_test = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_test.reshape(-1,1))
    
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [24]:
screen = (86,86)
GRID_SIZE = 4
CLASSES = 10

In [1]:
def create_img(data_array, label_array):
#     img = Image.new('F', screen)
    img = np.zeros(screen, dtype='float32')

    mnist_imgs = [Image.fromarray(d*255) for d in data_array]
    # mnist_img = mnist_img.crop((2,2,26,26))

    w_list = [ int(round(28 * np.random.triangular(0.5, 0.5, 1.5))) for i in range(3)]
    mnist_imgs = [np.array(x.resize((w,w), Image.BICUBIC), dtype='float32')/255 for x,w in zip(mnist_imgs, w_list)]

    x_list = [np.random.randint(0, screen[0]-w) for i, w in enumerate(w_list)]
    y_list = [np.random.randint(0, screen[1]-w) for i, w in enumerate(w_list)]
    for mimg, x, y, w in zip(mnist_imgs, x_list, y_list, w_list):
        img[y:y+w, x:x+w] += mimg
    
    img = img.clip(0,1).reshape(screen[0],screen[1],1)
    
    grid_x = screen[0]/GRID_SIZE
    grid_y = screen[1]/GRID_SIZE
    dw = 1./screen[0]
    for i, w in enumerate(w_list):
        x_list[i] += w/2
        y_list[i] += w/2
        w_list[i] *= dw
    dx = screen[0]/GRID_SIZE
    dy = screen[1]/GRID_SIZE
    
    trg = np.zeros((GRID_SIZE,GRID_SIZE,CLASSES+5), dtype='float32')
    for x, y, w, i in zip(x_list, y_list, w_list, range(3)):
        trg[ int(y//dy) ][ int(x//dx) ][:10] = label_array[i]
        trg[ int(y//dy) ][ int(x//dx) ][10:] = [(x / grid_x) - (x // grid_x), (y / grid_y) - (y // grid_y), w, w, 1]
    
    return img, trg

In [94]:
class DataGenerator():
    def __init__(self, data_array, label_array, batch_size=32):
        self.array = data_array
        self.label_array = label_array
        self.batch_size = batch_size
        
    def gen_batch(self):
        img_batch = []; trg_batch = []
#         idx = [np.random.randint(0, len(self.array)-self.batch_size) for i in range(self.batch_size)]
        for i in range(self.batch_size):
            idx = [np.random.randint(0, len(self.array)-self.batch_size) for i in range(3)]
            img, trg = create_img(self.array[idx], self.label_array[idx])
            img_batch.append(img); trg_batch.append(trg)
        return np.stack(img_batch), np.stack(trg_batch)
    
    def flow(self):
        while True:
            yield self.gen_batch()

a = DataGenerator(x_train, y_train).gen_batch()

i=0

plt.imshow(a[0][i].reshape(86,86), cmap='gray'); i+=1

a[1][i-1]

t, v, tt = load_data()
x_train, y_train = t

a[0].shape